<a href="https://colab.research.google.com/github/Sagar-11/21cs460_group20/blob/main/ML460_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Project Midway**
### Machine Translation under atypical use

# *Synopsis of papers*
---

## 1. Malinin, A., Band, N., Ganshin, Chesnokov, G., Gal, Gales, M., Noskov, A., Ploskonosov, A., Prokhorenkova, L., Provilkov, I., Raina, V., Raina, V., Roginskiy, D., Shmatova, M., Tigar, P., & Yangel, B. (2021). Shifts: A Dataset of Real Distributional Shift Across Multiple Large-Scale Tasks. arXiv preprint arXiv:2107.07455 .
---
* Recall: We want models that are robust to distributional shifts i.e they perform well on data 'in the wild' which could be very different than our training data
* Additionally we want it to give reliable uncertainty estimates on its prediction so we know we to trust it.
* Datasets: WMT'20en-ru, Newstest'19, WMT'19 MTNT reddit, Global voices, Shifts reddit.
* For predictive performance standard metrics can be used: BLEU, eGLEU,maxGLEU.
$$ eGLEU = \frac{1}{N}\sum_{i=1}^{N}\sum_{h=1}^{H}GLEU_{i,h}.w_{i,h}$$ , $$
\sum_{h=1}^{H}w_h = 1
$$
$$maxGLEU = \frac{1}{N}\sum_{i=1}^{N}max_h[GLEU_{i,h}]$$
* For the quality of uncertainity estimates we can use dicrimainatory power between 'indomain' and 'outdomain' using AUPR and AUROC but not a good idea.
* Instead authors propose area under error retention curves for joint estimates.
* Authors have used an ensemble of transformer-big trained using a fork of fairseq. They have used uncertainty estimates from a referenced paper (both discussed below)

|Data    | R-AUC   | F1-AUC  | ROC-AUC  |
|--------|---------|---------|----------|
| dev    |33.22±0.48| 0.428±0.003| 68.9±0.28|
  

##2. Vaswani, A., Shazeer, N., Parmar, N., Uszkoreit, J., Jones, L., Gomez, A. N., Kaiser, Ł. & Polosukhin, I. (2017). Attention is all you need. Advances in Neural Information Processing Systems (p./pp. 5998--6008), .
---
* Why do we need Transformers?
* Translation is autoregressive task where given an input sequence we must return an output sequence.
* RNNs are neural networks that roll out in discrete time steps and thus have been used in vec2seq, seq2vec, seq2seq tasks
* They suffer from a few major drawbacks: They are slow to train, Vanishing Gradients hence do not perform good for longer sentences.
* The LSTM block was introduced to solve the problem of vanishing gradients but they are even slower to train.
* The problem is their sequential nature that do not allow parallelisation
* The self-attention layers were added, with the idea that not all words are equally important for translation so only focus on some parts.
$$
\text { Attention }(Q, K, V)=\operatorname{softmax}\left(\frac{Q K^{T}}{\sqrt{d_{k}}}\right) V
$$
$$
\begin{aligned}
P E_{(p o s, 2 i)} &=\sin \left(p o s / 10000^{2 i / d_{\text {model }}}\right) \\
P E_{(\text {pos }, 2 i+1)} &=\cos \left(p o s / 10000^{2 i / d_{\text {model }}}\right)
\end{aligned}
$$
* Because of their significant improvements they have became the base for many other sequence prediction and NLP models. (BERT, GPT3)

![](https://drive.google.com/uc?export=view&id=1GfBo38rzx0jz0mMEDoNv9ByS_svC6NmL)

We will go through some other implementaion details later as many implementations are already available.
[Tensorflow tutorial on details of the Transformer model](https://www.tensorflow.org/text/tutorials/transformer#setup)
 


##3.  Malinin, A., & Gales, M. (2020). Uncertainty Estimation in Autoregressive Structured Prediction. arXiv preprint arXiv:2002.07650.
---
* Their goal is to give unsupervised uncertainty estimates in autoregressive tasks at both token and sentence level.
* The authors claim that there are two types of uncertainties Knowledge and Data
* Knowledge uncertainty captures the uncertainty due to the lack of understanding of data by the model.
* Data uncertainty is the intrinsic uncertainty in the data
* The authors take a bayesian approach, they assume that model parameter $\mathbf{\theta}$ is a random variable with a prior $p(\mathbf{\theta})$ 
* The predictive Posterior given the data $p(\mathbf{\theta}| \mathcal{D})$ is intractable so we take $q(\mathbf{\theta}) \approx p(\mathbf{\theta}| \mathcal{D}) $
* Using entropy chain rules we can write total uncertainity = know. uncertainty + data uncertainty
* Taking inspiration from this they use mutual information between y and $\theta$ to quantify ensemble diversity.
* Next they use EPKL and reverse mutual information to measure model diversity/uncertainty
* They explore a few properties of the three measures give monte carlo approximates to calculate them.
* They also explore how different ways of ensemble prediction effects uncertainty estimates.
$$
\mathrm{P}(\boldsymbol{y} \mid \boldsymbol{x}, \mathcal{D})=\mathbb{E}_{\mathbf{q}(\theta)}[\mathrm{P}(\boldsymbol{y} \mid \boldsymbol{x}, \boldsymbol{\theta})] \approx \frac{1}{M} \sum_{m=1}^{M} \mathrm{P}\left(\boldsymbol{y} \mid \boldsymbol{x}, \boldsymbol{\theta}^{(m)}\right), \boldsymbol{\theta}^{(m)} \sim \mathrm{q}(\boldsymbol{\theta}) \approx \mathrm{p}(\boldsymbol{\theta} \mid \mathcal{D})
$$
$$
\mathcal{H}[\mathrm{P}(\boldsymbol{y} \mid \boldsymbol{x}, \mathcal{D})]=\mathbb{E}_{\mathrm{P}(\boldsymbol{y} \mid \boldsymbol{x}, \mathcal{D})}[-\ln \mathrm{P}(\boldsymbol{y} \mid \boldsymbol{x}, \mathcal{D})]=-\sum_{\boldsymbol{y} \in \mathcal{Y}} \mathrm{P}(\boldsymbol{y} \mid \boldsymbol{x}, \mathcal{D}) \ln \mathrm{P}(\boldsymbol{y} \mid \boldsymbol{x}, \mathcal{D})
$$
$$
\underbrace{\mathcal{I}[\boldsymbol{y}, \boldsymbol{\theta} \mid \boldsymbol{x}, \mathcal{D}]}_{\text {Know. Uncertainty }}=\mathbb{E}_{\mathbf{q}(\boldsymbol{\theta})}\left[\mathbb{E}_{\mathrm{P}(\boldsymbol{y} \mid \boldsymbol{x}, \boldsymbol{\theta})}\left[\ln \frac{\mathrm{P}(\boldsymbol{y} \mid \boldsymbol{x}, \boldsymbol{\theta})}{\mathrm{P}(\boldsymbol{y} \mid \boldsymbol{x}, \mathcal{D})}\right]\right]=\underbrace{\hat{\mathcal{H}}[\mathrm{P}(\boldsymbol{y} \mid \boldsymbol{x}, \mathcal{D})]}_{\text {Total Uncertainty }}-\underbrace{\mathbb{E}_{\mathbf{q}(\boldsymbol{\theta})}[\hat{\mathcal{H}}[\mathrm{P}(\boldsymbol{y} \mid \boldsymbol{x}, \boldsymbol{\theta})]]}_{\text {Expected Data Uncertainty }}
$$

$$
\mathcal{K}[\boldsymbol{y}, \boldsymbol{\theta} \mid \boldsymbol{x}, \mathcal{D}]=\mathbb{E}_{\mathbf{q}(\boldsymbol{\theta}) \mathbf{q}(\tilde{\theta})}\left[\mathbb{E}_{\mathrm{P}(\boldsymbol{y} \mid \boldsymbol{x}, \boldsymbol{\theta})}\left[\ln \frac{\mathrm{P}(\boldsymbol{y} \mid \boldsymbol{x}, \boldsymbol{\theta})}{\mathrm{P}(\boldsymbol{y} \mid \boldsymbol{x}, \tilde{\boldsymbol{\theta}})}\right]\right] \quad \mathrm{q}(\boldsymbol{\theta}) \approx \mathrm{p}(\boldsymbol{\theta} \mid \mathcal{D})
$$

$$
\mathcal{M}[\boldsymbol{y}, \boldsymbol{\theta} \mid \boldsymbol{x}, \mathcal{D}]=\mathbb{E}_{\mathrm{q}(\boldsymbol{\theta})}\left[\mathbb{E}_{\mathrm{P}(\boldsymbol{y} \mid \boldsymbol{x}, \mathcal{D})}\left[\ln \frac{\mathrm{P}(\boldsymbol{y} \mid \boldsymbol{x}, \mathcal{D})}{\mathrm{P}(\boldsymbol{y} \mid \boldsymbol{x}, \boldsymbol{\theta})}\right]\right], \quad \mathrm{q}(\boldsymbol{\theta}) \approx \mathrm{p}(\boldsymbol{\theta} \mid \mathcal{D})
$$
$$
\hat{\mathcal{H}}_{\mathrm{C}-\mathrm{MC}}^{(S)}[\mathrm{P}(\boldsymbol{y} \mid \boldsymbol{x}, \mathcal{D})] \approx \frac{1}{S} \sum_{s=1}^{S} \frac{1}{L^{(s)}} \sum_{l=1}^{L^{(s)}} \mathcal{H}\left[\mathrm{P}\left(y_{l} \mid \boldsymbol{y}_{<l}^{(s)}, \boldsymbol{x}, \mathcal{D}\right)\right], \quad \forall \boldsymbol{y}_{<l}^{(s)} \subset \boldsymbol{y}^{(s)} \sim \mathrm{P}(\boldsymbol{y} \mid \boldsymbol{x}, \mathcal{D})
$$

## 4. Wang, Y., Wu, L., Xia, Y., Qin, T., Zhai, C., & Liu, T.-Y. (2020). Transductive Ensemble Learning for Neural Machine Translation. Proceedings of the AAAI Conference on Artificial Intelligence, 34(04), 6291-6298.
---

* While ensemble learning in effective in many scenarios it has some limitations in some cases.
* When there are strong single models, ensemble based techinques do not give much improvement
* Also in the case of many models ensemble techinques do not offer much
* The authors thus propose a technique based on transductive learning to address these issues. They claim that there idea performs better based on other ensemble techniques like Knowledge distillation.
* The key idea is to generate a synthetic dataset from the validation and test set and generating their translation from ensemble then fine tuning any single model on this dataset so as to minimise a loss function. Then choose the best model that performs well on the validation set.


* Paul Michel, Tatsunori B. Hashimoto, & Graham Neubig (2021). Modeling the Second Player in Distributionally Robust Optimization. ArXiv, abs/2103.10282.
* Peters, M., Neumann, M., Iyyer, M., Gardner, M., Clark, C., Lee, K., & Zettlemoyer, L. (2018). Deep contextualized word representations. In Proc. of NAACL.
* Zhang J, Mani I. KNN approach to unbalanced data distributions: a case study involving information extraction. In: Proceedings of the ICML’2003 workshop on learning from imbalanced datasets. 2003.
* Lucia Specia, Zhenhao Li, J. Pino, Vishrav Chaudhary, Francisco Guzmán, Graham Neubig, Nadir Durrani, Yonatan Belinkov, Philipp Koehn, Hassan Sajjad, Paul Michel, & Xian Li (2020). Findings of the WMT 2020 Shared Task on Machine Translation Robustness. In WMT.
---

# *Implementation and Details*
---



In [ ]:
 # Directory for Project
 %cd /content/drive/MyDrive/ML460\ Project

/content/drive/MyDrive/ML460 Project


In [ ]:
# downloads dataset, creates output dir wmt20_en_ru
!bash prepare_data.sh

In [ ]:
# converts some tsv files to csv, pre-process train data
# pre- process train data:- Normalise punctuation, non-print character removed
# and tokenise using Mosesdecoder. token files stored in wmt20../tmp
!bash prepare_data.sh

Cloning into 'mosesdecoder'...
remote: Enumerating objects: 148070, done.
remote: Counting objects: 100% (498/498), done.
remote: Compressing objects: 100% (206/206), done.
remote: Total 148070 (delta 315), reused 433 (delta 289), pack-reused 147572
Receiving objects: 100% (148070/148070), 129.86 MiB | 7.14 MiB/s, done.
Resolving deltas: 100% (114341/114341), done.
Checking out files: 100% (3489/3489), done.
Cloning into 'subword-nmt'...
remote: Enumerating objects: 580, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 580 (delta 0), reused 1 (delta 0), pack-reused 576
Receiving objects: 100% (580/580), 237.41 KiB | 3.39 MiB/s, done.
Resolving deltas: 100% (349/349), done.
pre-processing train data...
rm: cannot remove 'wmt20_en_ru/tmp/train.tags.en-ru.tok.en': No such file or directory
train-data/paracrawl-release1.en-ru.zipporah0-dedup-clean
Tokenizer Version 1.1
Language: en
Number of threads: 24
train-data/commoncrawl.ru

In [ ]:
#same as above for news dataset
!bash prepare_data.sh

processing back-trans news dataset
Tokenizer Version 1.1
Language: en
Number of threads: 24
Tokenizer Version 1.1
Language: ru
Number of threads: 24


In [ ]:
# remove duplications
!bash prepare_data.sh

Remove Deplications, clean data
prepare_data.sh: line 123: 20469 Broken pipe             paste $tmp/train.tags.$lang.tok.en $tmp/train.tags.$lang.tok.ru
     20470 Killed                  | python $CLEAN_DATA --directions en-ru --no-zero-len --max-sent-len 1024 --no-bad-utf --max-jaccard-coef-exclusive 0.05 --filter-equality >> $tmp/tmp


In [ ]:
# splitting train and valid
!bash prepare_data.sh

splitting train and valid...


In [ ]:
!bash prepare_data.sh

learn_bpe.py on wmt20_en_ru/tmp/train.en-ru...


In [ ]:
!bash prepare_data.sh

apply_bpe.py to train.en...
apply_bpe.py to valid.en...
apply_bpe.py to test19.en...
apply_bpe.py to reddit_dev.en...
apply_bpe.py to train.ru...


In [ ]:
!bash prepare_data.sh

apply_bpe.py to train.ru...
apply_bpe.py to valid.ru...
apply_bpe.py to test19.ru...
apply_bpe.py to reddit_dev.ru...


In [ ]:
#runs a script to clean and outs in prep/train and prep/valid
!bash prepare_data.sh

clean-corpus.perl: processing wmt20_en_ru/tmp/bpe.train.en & .ru to wmt20_en_ru/train, cutoff 1-250, ratio 1.5
..........(100000)..........(200000)..........(300000)..........(400000)..........(500000)..........(600000)..........(700000)..........(800000)..........(900000)..........(1000000)..........(1100000)..........(1200000)..........(1300000)..........(1400000)..........(1500000)..........(1600000)..........(1700000)..........(1800000)..........(1900000)..........(2000000)..........(2100000)..........(2200000)..........(2300000)..........(2400000)..........(2500000)..........(2600000)..........(2700000)..........(2800000)..........(2900000)..........(3000000)..........(3100000)..........(3200000)..........(3300000)..........(3400000)..........(3500000)..........(3600000)..........(3700000)..........(3800000)..........(3900000)..........(4000000)..........(4100000)..........(4200000)..........(4300000)..........(4400000)..........(4500000)..........(4600000)..........(4700000).....

In [ ]:
!bash prepare_data.sh

In [ ]:
# installs a fork of fairseq with uncertainity estimates
%cd structured-uncertainty
!python3 -m pip install --user --no-deps --editable .

/content/drive/MyDrive/ML460 Project/structured-uncertainty
Obtaining file:///content/drive/MyDrive/ML460%20Project/structured-uncertainty
  Running setup.py develop for fairseq


In [ ]:
%cd .. 

/content/drive/My Drive/ML460 Project


In [ ]:
# data into fairseq fmat for training and inference.
!python3 structured-uncertainty/preprocess.py  --srcdict baseline-models/dict.en.txt --tgtdict baseline-models/dict.ru.txt --source-lang en --target-lang ru \\
--trainpref wmt20_en_ru/train --validpref wmt20_en_ru/valid --testpref wmt20_en_ru/test19,wmt20_en_ru/reddit_dev  \\
--destdir data-bin/wmt20_en_ru --thresholdtgt 0 --thresholdsrc 0  --workers 24

Namespace(align_suffix=None, alignfile=None, all_gather_list_size=16384, bpe=None, cpu=False, criterion='cross_entropy', dataset_impl='mmap', destdir='data-bin/wmt20_en_ru', empty_cache_freq=0, fp16=False, fp16_init_scale=128, fp16_scale_tolerance=0.0, fp16_scale_window=None, joined_dictionary=False, log_format=None, log_interval=1000, lr_scheduler='fixed', memory_efficient_fp16=False, min_loss_scale=0.0001, no_progress_bar=False, nwordssrc=-1, nwordstgt=-1, only_source=False, optimizer='nag', padding_factor=8, seed=1, source_lang='en', srcdict='baseline-models/dict.en.txt', target_lang='ru', task='translation', tensorboard_logdir='', testpref='wmt20_en_ru/test19,wmt20_en_ru/reddit_dev', tgtdict='baseline-models/dict.ru.txt', threshold_loss_scale=None, thresholdsrc=0, thresholdtgt=0, tokenizer=None, trainpref='wmt20_en_ru/train', user_dir=None, validpref='wmt20_en_ru/valid', workers=24)
| [en] Dictionary: 43767 types
| [en] wmt20_en_ru/train.en: 29815305 sents, 873560493 tokens, 1.91% 

In [ ]:
# inference using single model
!bash single_model.sh

mkdir: cannot create directory ‘single’: File exists


In [ ]:
pip install sacrebleu

     |████████████████████████████████| 90 kB 5.8 MB/s 


In [ ]:
pip install sacremoses

     |████████████████████████████████| 895 kB 12.3 MB/s 


In [ ]:
pip install subword_nmt

In [ ]:
pip install nltk

In [ ]:
!bash shifts/translation/assessment/eval_single.sh

Traceback (most recent call last):
  File "shifts/translation/assessment/evaluate.py", line 391, in <module>
    main()
  File "shifts/translation/assessment/evaluate.py", line 338, in main
    refs_out, hypos_out, ids_out, nlls_out = load_text(args.path_out, beam_width=args.beam_width)
  File "shifts/translation/assessment/evaluate.py", line 304, in load_text
    nlls = -np.loadtxt(os.path.join(path, 'hypo_likelihoods.txt'), dtype=np.float32).reshape([-1, beam_width])
ValueError: cannot reshape array of size 3118 into shape (5)


In [ ]:
#trains model frm scrtch
!python3 structured-uncertainty/train.py data-bin/wmt20_en_ru --arch transformer_wmt_en_de_big --share-decoder-input-output-embed --fp16 --memory-efficient-fp16 --num-workers 16 --optimizer adam --adam-betas '(0.9, 0.98)' --clip-norm 0.0 --lr 5e-4 --lr-scheduler inverse_sqrt --warmup-updates 4000 --dropout 0.1 --weight-decay 0.0001 --criterion label_smoothed_cross_entropy --label-smoothing 0.1 --max-tokens 5120 --save-dir MODEL_DIR --max-update 50000 --update-freq 16 --keep-last-epochs 10 --seed 0

Namespace(activation_dropout=0.0, activation_fn='relu', adam_betas='(0.9, 0.98)', adam_eps=1e-08, adaptive_input=False, adaptive_softmax_cutoff=None, adaptive_softmax_dropout=0, all_gather_list_size=16384, arch='transformer_wmt_en_de_big', attention_dropout=0.1, best_checkpoint_metric='loss', bpe=None, bucket_cap_mb=25, clip_norm=0.0, clip_value=0, cpu=False, criterion='label_smoothed_cross_entropy', cross_self_attention=False, curriculum=0, data='data-bin/wmt20_en_ru', dataset_impl=None, ddp_backend='c10d', decoder_attention_heads=16, decoder_embed_dim=1024, decoder_embed_path=None, decoder_ffn_embed_dim=4096, decoder_input_dim=1024, decoder_layerdrop=0, decoder_layers=6, decoder_layers_to_keep=None, decoder_learned_pos=False, decoder_normalize_before=False, decoder_output_dim=1024, device_id=0, disable_validation=False, distributed_backend='nccl', distributed_init_method=None, distributed_no_spawn=False, distributed_port=-1, distributed_rank=0, distributed_world_size=1, dropout=0.1, 

In [ ]:
!python3 structured-uncertainty/interactive.py \
    --path baseline-models/model1.pt baseline-models/ \
    --beam 5 --source-lang en --target-lang ru \
    --tokenizer moses \
    --bpe subword_nmt --bpe-codes wmt20_en_ru/code \
    --compute-uncertainty

Namespace(all_gather_list_size=16384, beam=5, bpe='subword_nmt', bpe_codes='wmt20_en_ru/code', bpe_separator='@@', buffer_size=1, compute_uncertainty=True, cpu=False, criterion='cross_entropy', data='baseline-models/', dataset_impl=None, decoding_format=None, diverse_beam_groups=-1, diverse_beam_strength=0.5, empty_cache_freq=0, ensemble_sum_prod=False, force_anneal=None, fp16=False, fp16_init_scale=128, fp16_scale_tolerance=0.0, fp16_scale_window=None, gen_subset='test', input='-', iter_decode_eos_penalty=0.0, iter_decode_force_max_iter=False, iter_decode_max_iter=10, iter_decode_with_beam=1, iter_decode_with_external_reranker=False, left_pad_source='True', left_pad_target='False', lenpen=1, load_alignments=False, log_format=None, log_interval=1000, lr_scheduler='fixed', lr_shrink=0.1, match_source_len=False, max_len_a=0, max_len_b=200, max_sentences=1, max_source_positions=1024, max_target_positions=1024, max_tokens=None, memory_efficient_fp16=False, min_len=1, min_loss_scale=0.0001,

# Post Midway
---

* Work with multi30k, pytorch, fairseq libarary, fix errors
* Focus on uncertainty estimates, find other ways to jointly asses performance and uncertainty. 
* Report writing and Website maintenance. 